In [42]:
import sys
from adjmatrix import Vertex
from adjmatrix import Graph

class InefficientPrim:
    U, V = set (), set ()
    edges = set ()

    def __init__ (self, g):
        self.graph = g
        
    def find_mst (self, root):
        #initialize U and V
        self.U.add (root)
        for v in self.graph.vertices:
            if v != root:
                self.V.add (v)
        
        while (len(self.V) > 0):
            #iterate over edges (u,v) with u from U and v from V
            #keep track of minimum weight edge found
            min_u, min_v, min_weight = -1, -1, sys.maxsize
            for u in self.U:
                for v in self.V:
                    index_u = self.graph.edge_indices [u]
                    index_v = self.graph.edge_indices [v]
                    weight = self.graph.edges [index_u][index_v]
                    if weight < min_weight:
                        min_u, min_v, min_weight = u, v, weight
            #print (min_u, '--', min_v, ' : weight = ', min_weight)
            
            #add the edge (min_u, min_v) to the tree
            self.edges.add ((min_u, min_v))
            #move min_v from V to U
            self.V.remove (min_v)
            self.U.add (min_v)

In [54]:
g = Graph (True, True)
for i in range (ord('1'), ord('7')):
    g.add_vertex (Vertex(chr(i)))
    
edges = [('12', 6), ('13', 1), ('14', 5), ('23', 5), ('25', 3), ('34', 5), ('35', 6), ('36', 4), ('56', 1), ('46', 2)]
for edge in edges:
    g.add_edge (edge [0][:1], edge [0][1:], edge [1])

#g.print_graph ()


In [43]:

inefficient_prim = InefficientPrim (g)
inefficient_prim.find_mst ('4')
print (inefficient_prim.edges)

{('3', '1'), ('4', '6'), ('6', '3'), ('2', '5'), ('3', '2')}


In [55]:
class Prim:
    #resulting mcs tree
    mcs_tree = set ()
    #dictionary of vertex v (key) closest to which vertex u (value)
    closest_u = {}
    #dictionary of distance (value) from closest vertex v to vertex u (key)
    distance_from_u = {} 
    
    def __init__ (self, g):
        self.graph = g
        
    def init (self, root):
        #initialize the nearest U and distance from U dictionaries
        for v, i in self.graph.edge_indices.items ():
            #skip inserting the root
            if v == root:
                continue;
            
            self.closest_u [v] = root
            self.distance_from_u [v] = self.graph.edges [self.graph.edge_indices[root]][i]

    def find_mcs_tree (self, root):
        self.init (root)
        
        #while we still have vertices in V
        while (len (self.distance_from_u) > 0):
            #find an edge U-V with minimum distance
            min_v, min_dist = '', sys.maxsize
            for v, d in self.distance_from_u.items ():
                if d < min_dist:
                    min_v, min_dist = v, d
            
            #add an edge to the mcs tree
            self.mcs_tree.add ((self.closest_u [min_v], min_v))
            #remove v from the dictionaries
            self.closest_u.pop (min_v)
            self.distance_from_u.pop (min_v)
            
            #update the dictionaries based on min_v getting added to U
            min_v_index = self.graph.edge_indices [min_v]
            for col_index in range (0, len(self.graph.vertices)):
                distance_from_min_v = self.graph.edges [min_v_index][col_index]
                #if we find a vertex reachable from min_v
                if distance_from_min_v < sys.maxsize:
                    #get name of that vertex, given its index (the column in graph.edges)
                    update_candidate_vertex = self.graph.get_vertex_from_index (col_index)
                    #if that vertex is a member of V (i.e. not a member of U)
                    if (update_candidate_vertex in self.closest_u):
                        #if its existing distance from a vertex in U is higher than distance_from_min_v
                        if self.distance_from_u [update_candidate_vertex] > distance_from_min_v:
                            #update the dictionaries to reflect the new closest vertex and distance
                            self.closest_u [update_candidate_vertex] = min_v
                            self.distance_from_u [update_candidate_vertex] = distance_from_min_v
                #for v_index in self.graph.edges [min_v_index]:
                

In [56]:
prim = Prim (g)
prim.find_mcs_tree ('1')
print (prim.mcs_tree)

{('5', '2'), ('1', '3'), ('3', '6'), ('6', '4'), ('6', '5')}
